# **xSoc Python Course** - Week 6

### *Data Processing*

🖋️ *Written by Alistair & Laura from the [Warwick AI](https://warwick.ai)*

---

## DataFrames and Indexing
In this section, we will cover the first step of any data processing task, importing the data. Previously in week 4 we looked at how to read in CSV data, however that method was quite cumbersome. Fortunately, Python has a library called `pandas` that can make importing and processing our data a lot easier. 

In [2]:
import pandas

Now we have imported pandas, we can load in our csv data into a `DataFrame` object. DataFrames are a way of storing various columns of data with a shared index. In this first example, we will create three rows of data and index each row with a letter.

In [3]:
row_1 = [1,9,7]
row_2 = [3,5,4]
row_3 = [6,8,2]
df = pandas.DataFrame([row_1,row_2,row_3], index=['A','B','C'])
df

,0,1,2
A,1,9,7
B,3,5,4
C,6,8,2


We can see that the DataFrame is output with the two rows, indexed by the letters we gave. If we change the index letters around, then we will create a different DataFrame with different indexing.

Now we have a DataFrame, we can access the data within it using indexing. Let's get the first column of data from the DataFrame.

In [4]:
df[1]

A    9
B    5
C    8
Name: 1, dtype: int64

Now let's get the first row of data (row 'A'). Note this time we need to use `.loc[]` as this indicates that we want to index using the row labels.

In [5]:
df.loc['A']

0    1
1    9
2    7
Name: A, dtype: int64

We can also index a row by the postition of the row by using `.iloc[]`, for example we can get the second row of data as follows. (Remember we need to use 1 as array indexing starts from 0)

In [6]:
df.iloc[1]

0    3
1    5
2    4
Name: B, dtype: int64

Lastly, we can also perform conditional indexing where we specific a condition for the indexing, for example lets return a DataFrame which only contains values >= 5.

In [7]:
df>=5

,0,1,2
A,False,True,True
B,False,True,False
C,True,True,False


In [8]:
df[df>5]

,0,1,2
A,NaN,9.0,7.0
B,NaN,NaN,NaN
C,6.0,8.0,NaN


Lastly, we can also change the index to something else if we want. For example, let's change from `A,B,C` to `X,Y,Z`.

In [9]:
new_index = ['X','Y','Z']
df['Index'] = new_index
df.set_index('Index')

,0,1,2
Index,,,
X,1,9,7
Y,3,5,4
Z,6,8,2


# Motivating Example

Now we have a basic idea of how dataframes work, in this section we will go through an example data processing task which will show various techniques we can use to analyse data and even create a machine learning model from it!

We will look at a dataset containing data about customers of an ecommerce platform. We will import this dataset into Python, clean it up, apply some processing to the dataset, visualise the dataset and analyse trends, and then we will predict the values of new data points based on it. This should hopefully mimic a real life data processing scenario and give you some idea of the process and techniques to follow and use.

## Importing Data

Now we now how to access data within a DataFrame, lets look at how to import external data into a DataFrame. In a lot of cases, the data we want to process will be stored as either `.csv` files or Excel Spreadsheets. Fortunately, pandas can work with both!

To import data from a CSV file, we can use the `.read_csv()` function as follows. In this instance we will import the `Ecommerce_Customers.csv` file.

In [21]:
ecommerce_data = pandas.read_csv('.\\Data\\Ecommerce_Customers.csv')
ecommerce_data.head()

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer Common\nDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582\nCobbborough, D...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505
3,riverarebecca@gmail.com,"1414 David Throughway\nPort Jason, OH 22070-1220",SaddleBrown,34.305557,13.717514,36.721283,3.120179,581.852344
4,mstephens@davidson-herman.com,"14023 Rodriguez Passage\nPort Jacobville, PR 3...",MediumAquaMarine,33.330673,12.795189,37.536653,4.446308,599.406092


If you are working with a dataframe saved as an Excel spreadsheet, you can use the `.read_excel()` function to achieve the same effect. Note this function takes an argument `sheet_name` to indicate which sheet in the Excel workbook to import.

In [22]:
excel_dataframe = pandas.read_excel('.\\Data\\excel_data.xlsx',sheet_name='Sheet1')
excel_dataframe

,A,B,C,D
0,1,4,2,3
1,6,3,1,4
2,4,8,3,5


## Data Cleaning

Now that we have the data imported, we can start by cleaning the data to make it easier to work with. This is a common practice for data processing operations as sometimes the data you will be working with will have missing values or information that is harder to process and understand.

First we will use the `.describe()` function to understand different attributes of the data. 

In [23]:
ecommerce_data.describe()

,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,33.053194,12.052488,37.060445,3.533462,499.314038
std,0.992563,0.994216,1.010489,0.999278,79.314782
min,29.532429,8.508152,33.913847,0.269901,256.670582
25%,32.341822,11.388153,36.349257,2.930450,445.038277
50%,33.082008,11.983231,37.069367,3.533975,498.887875
75%,33.711985,12.753850,37.716432,4.126502,549.313828
max,36.139662,15.126994,40.005182,6.922689,765.518462


---

#### **Congrats on making it to the end of the Cross-Society Python Course!**

Thanks so much for taking part, and we hope you found it useful.

---

# Bonus Tasks

🖋️ ***This week was written by Alistair and Laura from the [Warwick AI](https://warwick.ai)***

We hope you enjoyed the course!